In [6]:
import numpy as np
# importing decimal module to control significant figures
import decimal

In [7]:
def toDecimal(array) :
    # flatten the array convert floats to decimal and reshape to original shape
    originalShape = array.shape
    array = array.flatten().astype(object) 
    for i in range(array.size) :
        array[i] = decimal.Decimal(str(array[i]))
    array = array.reshape(originalShape)   
    return array


def intializeContext(significantFigs,rounding) :
    #Intialize the context with specified signFigs and rounding/chopping
    decimal.getcontext().prec = significantFigs 
    if( rounding ) :
        decimal.getcontext().rounding = decimal.ROUND_HALF_UP
    else :
        decimal.getcontext().rounding = decimal.ROUND_DOWN 
        
def toFloats (array) :
    #Convert the decimals back to float
    originalShape = array.shape
    array = array.flatten().astype(object)
    for i in range(array.size) :
        array[i] = float(array[i]) 
    array = array.reshape(originalShape)    
    return array            
    

In [8]:
def isDiagonalyDominant(A) :
    #Check if Matrix A is diagonaly dominant 
    dim = A.shape
    # Flags to check system
    at_least_one_strictly_greater = None 
    if(dim[0] != dim[1]) : raise Exception("Matrix must be square")
    for i in range(dim[0]) :
        nonDiagonalSum = 0 
        for j in range(dim[0]) :
            if(j==i) : continue 
            nonDiagonalSum += abs(A[i][j]) 
        if(A[i][i] > nonDiagonalSum) :
            at_least_one_strictly_greater = True
        elif(A[i][i] == nonDiagonalSum) :
            if(at_least_one_strictly_greater == None) : at_least_one_strictly_greater = False
        else :
            return False    
    return at_least_one_strictly_greater            
            

In [9]:
# GaussSeidel with relaxation for improved convergence
# the parameter n can be removed as we can get the no of vars from the npArray
def GaussSeidel_noNorm(A,b,n,x,maxIterations,ErrorTolerance,relax , significantFigs = 7 , rounding = True) :
    # Setting up signifcant figs and rounding/chopping
    intializeContext(significantFigs,rounding)
    
    #converting floats to decimals 
    A = toDecimal(A)
    b = toDecimal(b)
    x = toDecimal(x)
    relax = decimal.Decimal(str(relax))
    
    #List to hold iteration details
    iteration_details = []
    #Check for diagonaly dominant matrix
    if(isDiagonalyDominant(A) ) :
        isDominant = 'The matrix is diagonaly dominant'
    else :
        isDominant = 'The matrix is not diagonaly dominant'    
    # Calculating first iteration before applying relaxation    
    for i in range(n) :
        sum = b[i]
        for j in range(n) :
            if(i==j) :
                continue
            sum -= A[i][j] * x[j]
        x[i] = sum/A[i][i]
    details = {
            'iteration' : 1,
            'xNew' : toFloats(x),
            'maxError' : '_'
        }     
    iteration_details.append(details)
    iteration = 2
    # Loop until convergence or max iterations reached 
    while (True) :
        belowTolerance = True
        maxError = 0
        for i in range(n) :
            oldX = x[i]
            sum = b[i]
            for j in range(n) :
                if(i==j) :
                    continue
                sum -= A[i][j] * x[j]
            x[i] = relax*sum/A[i][i] + (1-relax)*oldX
            if (belowTolerance and x[i] != 0) :
                estimatedError = abs(float((x[i]-oldX)/x[i])) * 100
                estimatedError = float(estimatedError)
                if(estimatedError > ErrorTolerance):
                    belowTolerance = False
                maxError = max(maxError, estimatedError)
        details = {
            'iteration' : iteration,
            'xNew' : toFloats(x),
            'maxError' : float(maxError)
        }            
        iteration+=1
        iteration_details.append(details)
        if(belowTolerance or iteration >= maxIterations) :
            break
    lines = [isDominant]
    for d in iteration_details :
        s = (
            f"Iteration: {d['iteration']}\n"
            f"  xNew: {d['xNew']}\n"
            f"  maxError: {d['maxError']}"
        )
        lines.append(s)    
    return toFloats(x), lines

In [10]:
# Example linear system
A = np.array([[10, -1, 2, 0],
              [-1, 11, -1, 3],
              [2, -1, 10, -1],
              [0, 3, -1, 8]], dtype=float)

b = np.array([6, 25, -11, 15], dtype=float)

# Initial guess
x0 = np.zeros(len(b))

# Run Gauss-Seidel with relaxation
solution, details = GaussSeidel_noNorm(A, b, 4, x0, maxIterations=50, ErrorTolerance=1e-6, relax=0.8)

print("Solution:", solution)
print("\nIteration Details:")
print("\n".join(details))
# for d in details:
#     max_error = d['maxError']
#     # Ensure maxError is a float, else print as-is
#     if isinstance(max_error, (float, int)):
#         max_error_str = f"{max_error:.6f}"
#     else:
#         max_error_str = str(max_error)
#     print(f"Iteration {d['iteration']}: x = {d['xNew']}, maxError = {max_error_str}")


Solution: [1.0 2.0 -1.0 1.0]

Iteration Details:
The matrix is diagonaly dominant
Iteration: 1
  xNew: [0.6 2.327273 -0.9872727 0.8788635]
  maxError: _
Iteration: 2
  xNew: [0.9441456 2.088749 -0.9911087 0.9500371]
  maxError: 36.45048
Iteration: 3
  xNew: [0.9945067 2.028898 -0.9990273 0.9814353]
  maxError: 5.063928
Iteration: 4
  xNew: [1.001057 2.009978 -1.000661 0.9932276]
  maxError: 0.6543384
Iteration: 5
  xNew: [1.001115 2.003507 -1.000571 0.9975363]
  maxError: 0.00579354
Iteration: 6
  xNew: [1.000595 2.001241 -1.000307 0.9991043]
  maxError: 0.05196908
Iteration: 7
  xNew: [1.000268 2.000441 -1.000141 0.9996745]
  maxError: 0.032691239999999996
Iteration: 8
  xNew: [1.000111 2.000157 -1.000059 0.9998819]
  maxError: 0.01569826
Iteration: 9
  xNew: [1.000045 2.000056 -1.000024 0.9999572]
  maxError: 0.006599703
Iteration: 10
  xNew: [1.000017 2.00002 -1.000009 0.9999845]
  maxError: 0.002799952
Iteration: 11
  xNew: [1.000007 2.000008 -1.000003 0.9999942]
  maxError: 0.0009